In [36]:
# %pip install pycountry
# %pip install fuzzywuzzy

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import pycountry
from fuzzywuzzy import process
import time


# Introduction to the dataset and what I'll be keeping
Hi there, this project is mostly a tableau focused one but this script will explain how I processed, cleaned, and repackaged the originally dirty data to a nicer form that tableau can easily use. The data itself comes from a websurvey mostly answered at the end of April 2021, accessible [here](https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html). I'll walk through the columns, briefly explain what the data in them looks like, and explain my plan for them going forward. 

There are 28k entries in this csv with 18 features:
1. Timestamp: the data and time of the survey response. Won't use, most of the responses are within a few days of each other so not very interesting
2. How old are you?: ordinal but stored as the value of response (i.e. 25-34), will use after conversion back to ordinal
3. What industry do you work in?: ordinal again but stored as the value of the reponse. Will convert back to ordinal and use. For now I will toss out Other
4. Job Title: free form string, will search for some keywords to see if I can get some measure of seniority from it
5. Job Title Context: free form string, won't use, people used this to give additional context to their job title and industry but it's often not responded to and varies widely in response quality
6. Annual salary: free form string but generally people seemed to do it right, some gave it in units of kUSD so I'll convert that back to USD. Any non USD currencies I'll be using the exchange rate from the eurofxref.csv file in data to convert it to USD.
7. Additional Salary: same as the annual salary, will add them together eventually
8. Currency: string that expresses the type of currency salary expressed in
9. Currency Context: allows for expansion on the other response to currency, will ignore these to keep things simple
10. Additional Compensation context: won't use, few responses and highly variable
11. Country: free form string, will have to normalize after grouping the common terms for the same country (i.e. USA, US, usa, United States, etc.)
12. State: ordinal choice, the 50 states of the United States
13. What city do you work in: free form string but generally people answered it well. For this one I'm thinking of using the unique set and using that to determine which ones to keep. Big issues around mispellings and alternative expressions in this one (i.e. LA and Los Angeles)
14. Years of professional exp: ordinal but stored as the value of the response again, will use after conversion
15. Years of professional exp in the claimed industry: ordinal but stored as the value of the response again, will use after conversion
16. Highest level of education completed: same ordinal as before, will use
17. Gender: same ordinal as before, will use
18. Race: this allowed for multiple responses, I think I'll keep the non white one if they answered white and another otherwise I'll just take the first one. I'll try to see if tableau could handle multiple responses well

I'll rename the columns to something shorter since right now they're all the questions of the fields they represent and are quite long then drop all the ones I won't be using at all.

In [38]:
df = pd.read_csv('data/form_rep.csv')


In [39]:
df = df.set_axis(['Datetime', 
'Age', 
'Industry', 
'Job Title', 
'Job Title Context', 
'Salary', 
'Salary Additional', 
'Currency', 
'Currency Context', 
'Salary Context', 
'Country', 
'State', 
'City', 
'Years Total', 
'Years Specific', 
'Education', 
'Gender', 
'Race'], axis = 1)
drop_list = ['Datetime', 'Job Title Context', 'Salary Context', 'Currency Context']
df.drop(columns = drop_list, inplace=True)

The first thing I want to do is do the salary conversion. I'm using the exhange rates I got from the european bank database today 6/10/2024 and will use that to convert the salaries to USD. I'll be dropping the 120 entries that use other to keep things simple and changing the AUD/NZD entry to just NZD. Then I'll use the conversion function to change the salaries to USD after changing any entries that are three digits or less (assuming that's supposed to be kCurrency intead of Currency) to the proper units.

In [40]:
df.drop(df[df['Currency'] == 'Other'].index, inplace=True)
df.loc[df['Currency'] == 'AUD/NZD', 'Currency'] = 'NZD'

In [41]:
def convert_sals(x):
    if len(x)<=3:
        return int(x.replace(',', ''))*1000 #likely inputted in kUnits of money
    else:
        return int(x.replace(',', ''))
    
df['Salary'] = df['Salary'].apply(convert_sals) #salary needs conversion, salary additional is already a float. Can't assume units for it either
df.loc[df['Salary Additional'].isna(), 'Salary Additional'] = 0 #filling in the NaNs with 0s
df['Salary'] = df['Salary'] + df['Salary Additional']
df.drop(columns=['Salary Additional'], inplace=True)

In [42]:
def load_conv_df():
    conv_df = pd.read_csv('data/eurofxref.csv').rename(columns=lambda x: x.strip()) #some of the currencies end up have leading whitespaces otherwise
    conv_df.insert(0, 'EUR', 1)
    return conv_df

def convert_to_usd(amount, cur_str, conv_df):
    return int((amount * conv_df['USD'].values / conv_df[cur_str].values)[0])

conv_df = load_conv_df()
df['Salary'] = df.apply(lambda row: convert_to_usd(row['Salary'], row['Currency'], conv_df), axis=1)
df['Currency'] = 'USD' #we've done the conversion so now all are in USD

Next I'm going to tackle the industry and job title sections. First for industry as I said I'll be dropping any of the fill in your own ones since the choices were already expansive. We lose about 1000 entries for doing this, we're now at 25800 just about. 

In [43]:
valid_industry_list = ['Accounting, Banking & Finance',
'Agriculture or Forestry',
'Art & Design',
'Business or Consulting',
'Computing or Tech',
'Education (Primary/Secondary)',
'Education (Higher Education)',
'Engineering or Manufacturing',
'Entertainment',
'Government and Public Administration',
'Health care',
'Hospitality & Events',
'Insurance',
'Law',
'Law Enforcement & Security',
'Leisure, Sport & Tourism',
'Marketing, Advertising & PR',
'Media & Digital',
'Nonprofits',
'Property or Construction',
'Recruitment or HR',
'Retail',
'Sales',
'Social Work',
'Transport or Logistics',
'Utilities & Telecommunications']
df = df[df['Industry'].isin(valid_industry_list)]

For the senority terms I've defined a few levels using common terms I found scrolling around on the dataframe. It's not exhaustive but it should be alright. The default will be the mid level. After running it we get about 14k entries in the mid level with the rest distributed in the other levels. Pretty good for a quick estimate of seniority. Note that since we go down the list if someone has multiple titles that match we'll end up assigning them to the highest level (i.e. there's one entry that is Associate Senior Director and that goes to Executive Level as I think it should). Obviously the difference in job titles between industries is an issue here and if it were a bigger project it'd be worth really breaking it down.

In [44]:
#mid level will be the default so left off
senority_terms = { 'Junior Level': ['associate', 'jr', 'jr.', 'junior', 'intern'],
                    'Senior Level' : ['senior', 'sr', 'sr.'],
                    'Managerial Level' : ['lead', 'manager', 'head'],
                    'Executive Level' : ['president', 'ceo', 'executive', 'director']}
df['Senority'] = 'Mid Level'
for senior_label, senior_list in senority_terms.items():
    df.loc[df['Job Title'].str.contains('|'.join(senior_list), case=False), 'Senority'] = senior_label

Alright, moving on to some demographical stuff. Education and Race.

Education first, there are 200 missing entries in it. The options didn't allow for people who didn't finish high school to answer that but looking at the job titles of the people who didn't answer education I can safely conclude they did finish high school so I'll drop those 200 (some of the titles included Attorneys, Professors, and Directors of Hospitals, job titles that need a high education, these are the majority of them)

In [45]:
df.drop(df[df['Education'].isna()].index, inplace=True)

Now for race, 122 entries didn't answer so those will be dropped. The rest need to be simplified down to a single race to make filtering in tableau easier. The most common multi-race choice was something and white. We can just take the something since white dominates the set and we're interested in the contrast for this feature. There are a small handful of hybrid choices that don't include white. For that I'll just take the first one.

In [46]:
df.drop(df[df['Race'].isna()].index, inplace=True)

non_white_race_options = ['Asian or Asian American',
                'Black or African American',
                'Hispanic, Latino, or Spanish origin',
                'Middle Eastern or Northern African',
                'Native American or Alaska Native',
                'Another option not listed here or prefer not to answer']
def simplify_race(race_str, non_white_race_options):
    for nwr_option in non_white_race_options:
        if nwr_option in race_str:
            return nwr_option
    return 'White'

df['Race'] = df['Race'].apply(lambda x: simplify_race(x, non_white_race_options))

Now for the hardest part, the location. I'm going to keep the state as is for now and see how tableau handles the nans. State was chosen from a checklist so that's easy. For country though, I'm going to use a country list from pycountry which compiles the ISO country list and I'll use fuzzy string matching to try and get past typos. I ran this a few times and compiled the most common mistakes neither system handled right into the manual fixes to try and get as much good data through as I could. Some where unrecoverable due to respondants adding too much detail but that lost only a few hundred results

In [47]:
doge = pycountry.countries

In [48]:
country_names = [country.name for country in pycountry.countries]
country_a3_names = [pycountry.countries.get(name = country).alpha_3 for country in country_names]

In [49]:
def simplify_country(country_str, country_dict_list, manual_fixes):
    try:
        closest_match = process.extractOne(country_str, list(manual_fixes.keys()), score_cutoff=85)
        if closest_match:
            status_message = f'Input: {country_str} found a match on the manual override dicts: {manual_fixes[closest_match[0]]}'
            # print(status_message)
            return manual_fixes[closest_match[0]], status_message, country_str
        country_match = pycountry.countries.search_fuzzy(country_str)[0] #searches within subdivisions of countries as well so it's a good first check
        status_message = f'Input: {country_str} found a match on the pyc fuzzy search: {country_match.name}'
        # print(status_message)
        return country_match.name, status_message, country_str
    except:
        for country_dict in country_dict_list:
            closest_match = process.extractOne(country_str, list(country_dict.keys()), score_cutoff=85)
            if closest_match:
                status_message = f'Input: {country_str} found a match on the dicts: {country_dict[closest_match[0]]}'
                # print(status_message)
                return country_dict[closest_match[0]], status_message, country_str
        status_message = f'Input: {country_str} found no matches'
        # print(status_message)
        return 'ERROR', status_message, country_str #if we couldn't find any match we nan out the country for removal

# df['Country'] = df['Country'].apply(lambda x: simplfy_country(x))
manual_fixes_keys = ['US', 'ISA', 'UK', 'New Zealand', 'Australia', 'NA', 'Vietnam', 'Denmark', 'United States of America', 'America']
manual_fixes_values = ['United States', 'United States', 'United Kingdom', 'New Zealand', 'Australia', 'United States', 'Vietnam', 'Denmark', 'United States', 'United States']

manual_fixes = dict(zip(manual_fixes_keys, manual_fixes_values))
country_a3_2_name = {a3_name : countryn for countryn, a3_name in zip(country_names, country_a3_names)}
country_name_2_name = {countryn : countryn for countryn in country_names}


country_dicts = [country_name_2_name, country_a3_2_name]

# for cd in country_dicts:
df['Country'] = df['Country'].apply(lambda x: "".join(x for x in x if (x in string.ascii_letters) or (x==' '))) #cleaning any non letters/spaces out first


In [50]:
# import dask.dataframe as dd

# # Convert the pandas DataFrame to a Dask DataFrame
# ddf = dd.from_pandas(df, npartitions=8)

# # Define a Dask delayed function for simplify_country
# from dask import delayed

# @delayed
# def delayed_simplify_country(country_str, country_dict_list):
#     return simplify_country(country_str, country_dict_list)

# # Apply the delayed function to the Dask DataFrame
# ddf['Country'] = ddf['Country'].apply(lambda x: delayed_simplify_country(x, country_dicts), meta=('x', 'object'))

# # Compute the Dask DataFrame to get the result
# df = ddf.compute()

In [51]:
# doge = df['Country'].values
# doge2 = doge[0]


In [52]:
# doge2.compute()

In [53]:
test_cases = []
total_in = len(df['Country'].values)
st = time.time()
print(st)
for xind, x in enumerate(df['Country'].values):
    mean_rate = (time.time()-st)/(xind+1)
    print(f'\r{xind}/{total_in}, ETA: {mean_rate*(total_in-xind)/60:.2f} min', end='', flush=True)
    test_cases.append(simplify_country(x, country_dicts, manual_fixes))
# test_cases = [simplify_country(x, country_dicts) for x in df['Country'].values]
# df['Country'] = df['Country'].apply(lambda x: simplify_country(x, country_dicts))


1718078181.3550684
20274/25335, ETA: 0.18 min

21746/25335, ETA: 0.13 min

24079/25335, ETA: 0.04 min

25334/25335, ETA: 0.00 min

In [54]:
# doge = np.array(test_cases)
# failed_cases = []
# for tc in test_cases:
#     if tc == np.nan:
#         failed_cases.append(fc)

In [55]:
# cm_stack=[]
# cs = 'United kindom'
# for country_dict in country_dicts:
#     closest_match = process.extractOne(cs, list(country_dict.keys()), score_cutoff=80)
#     if closest_match:
#         cm_stack.append((cs, country_dict[closest_match[0]]))
#         break

In [56]:
failed_cases= [x for x in test_cases if x[0] == 'ERROR']

In [57]:
doge = np.array(test_cases)
# unique_count = set(doge[:,0])

I'm going to drop city since it won't add enough to be worth the trouble of ironing out the quality of that field. The countries took a good bit enough. I'll also drop the job title since the information from them are encoded into senority and the industry. At least for a high level look.

With that, we have the dataframe ready to go for tableau so we drop a save and head over to the dashboard.

In [58]:
df.drop(columns = ['Job Title', 'City'], inplace=True)
df.to_csv('data/cleaned_data.csv')